# 06 — Walk-Forward Evaluation
Evaluate ARIMA-only vs Hybrid ARIMA+LSTM using time-ordered splits (no leakage).

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import tensorflow as tf
from tensorflow.keras.models import load_model

corr_df  = pd.read_parquet("../data/processed/rolling_corr_sample.parquet")
orders_df = pd.read_csv("../data/processed/arima_orders_sample.csv")
resid_df = pd.read_parquet("../data/processed/arima_residuals_sample.parquet")
pred_df  = pd.read_parquet("../data/processed/arima_pred_sample.parquet")

In [ ]:
# One-series walk-forward
def walk_forward_arima_lstm(corr_series, arima_order, lstm_model, lookback=20, horizon=1):
    y = pd.Series(corr_series).dropna().astype("float32")
    n = len(y)

    # split indices
    train_end = int(0.7*n)
    val_end = int(0.85*n)

    y_train = y.iloc[:train_end]
    y_test  = y.iloc[val_end:]  # true test

    # Fit ARIMA on train, forecast over test period (dynamic refit omitted for simplicity)
    arima = sm.tsa.ARIMA(y_train, order=arima_order).fit()
    # one-step forecasts for test range
    arima_fc = arima.predict(start=y.index[val_end], end=y.index[-1])

    # Residuals on train for LSTM training signal
    arima_pred_in = arima.predict(start=y_train.index[0], end=y_train.index[-1])
    resid_train = (y_train - arima_pred_in).values

    # Create residual sequences for LSTM "state"
    def seq_last(resid_arr, t):
        # last lookback residuals ending at t-1
        if t < lookback:
            return None
        return resid_arr[t-lookback:t]

    # For a clean demo: use LSTM to forecast residual as 0 (if not enough history)
    # Better version: retrain LSTM per walk-forward window (expensive).
    # Here: just use the pretrained model passed in.
    resid_fc = []
    resid_hist = resid_train.copy()

    for step in range(len(arima_fc)):
        # build input from latest residual history
        if len(resid_hist) >= lookback:
            x_in = resid_hist[-lookback:].reshape(1, lookback, 1)
            rhat = float(lstm_model.predict(x_in, verbose=0).reshape(-1)[0])
        else:
            rhat = 0.0
        resid_fc.append(rhat)

        # update residual history with realized residual using observed y if available
        # approximate residual = y - arima_fc
        # (walk-forward: you "observe" next point sequentially)
        y_obs = float(y_test.iloc[step])
        r_real = y_obs - float(arima_fc.iloc[step])
        resid_hist = np.append(resid_hist, r_real)

    resid_fc = np.array(resid_fc, dtype="float32")
    hybrid_fc = arima_fc.values.astype("float32") + resid_fc

    y_true = y_test.values.astype("float32")

    mse_arima = float(np.mean((y_true - arima_fc.values)**2))
    mse_hybrid = float(np.mean((y_true - hybrid_fc)**2))
    mae_arima = float(np.mean(np.abs(y_true - arima_fc.values)))
    mae_hybrid = float(np.mean(np.abs(y_true - hybrid_fc)))

    return {"mse_arima": mse_arima, "mse_hybrid": mse_hybrid,
            "mae_arima": mae_arima, "mae_hybrid": mae_hybrid}

In [ ]:
# Run evaluation on one series
series = resid_df.columns[0]
row = orders_df[orders_df["series"] == series].iloc[0]
order = (int(row["p"]), int(row["d"]), int(row["q"]))

lstm = load_model(f"../models/lstm_residual_{series}.keras")

out = walk_forward_arima_lstm(corr_df[series], order, lstm, lookback=20)
out